# WEEK1      
> ## 数学符号     

$x^{<t>},y^{<t>}$代表输入输出的序列第$t$个时刻的位置    
$x^{(i)<t>},y^{(i)<t>}$代表输入输出的序列第$i$个样本第$t$个时刻位置的值      
$T_x,T_y$输入输出序列的长度    
$T_x^{(i)},T_x^{(i)}$代表第$i$个训练样本输入/输出序列的长度

> ## NLP      

NLP的做法：      
变量的每个维度代表一个句子的一个词：     
1. 做一张词表/词典，包含你需要使用的词语（常用词）    
2. 对输入序列One-hot编码（词典长度\*序列长度）    
3. 如果遇到不在词典中的词语——创作一个虚拟词汇（UNK,Unknown）     
<img style="float: center;" src="course_5_pics/WEEK_1_1.PNG" width=600 height=600>     

> ## 循环神经网络    

### 传统神经网络面对序列识别的问题
1. 输入序列非定长，填充不是一个好办法(pad会假设一个最大的文本长度)    
2. 没有学习到不同位置的输入(比如文本不同位置的意思)的意义             
    假设$x^{<1>}$处学到一个词的意思(比如断句),但当该词汇出现在$x^{<t>}$处，那么没办法识别到    
    
### 循环神经网络的基本工作    

- 初始时刻输入一个激活值（通常是0）    
从左到右读取一个句子（序列），那么这个第一个词的结构是$x^{<1>}$，当再读取$x^{<2>}$时会引入$x^{<1>}$的信息             
<img style="float: center;" src="course_5_pics/WEEK_1_2.PNG" width=600 height=600>  
$W_{ax}$代表$x$到隐藏层的连接的一系列参数,循环连接/水平链接$W_{aa}$，输出链接$W_{ya}$    

**从左到右扫描序列，所有的参数也是共享的**    

**问题**：只考虑到之前的信息，没有考虑到之后的信息，$x^{<t>}$只考虑了$\cdots,x^{<t-1>}$，没有考虑$x^{<t+1>}\cdots$       

### 循环神经网络前向传播       
1. **初始化激活向量$a^{<0>}= \vec{0}$**    


2. **$a^{<t>} = g(w_{aa}a^{<t-1>}+w_{ax}x^{<t>}+b_a)=g(w_{a}[a^{<t-1>},x^{<t>}]^{T}+b_a)$**  
    
    其中$w_a = [w_{aa},w_{ax}]$

<img style="float: center;" src="course_5_pics/WEEK_1_3.PNG" width=600 height=600> 
3. $\hat y^{<t>} = g(w_{ya}a^{<t>}+b_y)$      

    - 对于输出函数    
        二分类——Sigmoid    
        多分类——Softmax     
        
> ## 循环神经网络的反向传播          

**单个时间步上的Lost function**：
$$L^{<t>}(\hat y^{<t>},y^{<t>}) = -y^{<t>}log\hat y^{<t>}-(1-y^{<t>})log(1-\hat y^{<t>})$$     
**总的损失函数：所有单个时间步损失函数的求和**
$$L(\hat y,y) = \sum ^{T_x}_{t=1}L^{<t>}(\hat y^{<t>},y^{<t>})$$     

<img style="float: center;" src="course_5_pics/WEEK_1_4.PNG" width=600 height=600>      

> ## 不同类型的循环神经网络    

**RNN的类型：**   
- 每个时间节点输入输出——多对多（或者全部输入结束后输出多个）    
- 最后一个时间节点，每个时间节点输入——多对一    
- 第一个时间节点输入，每个时间节点输出——一对多      

对于机器翻译,输入或者输出的长度$T_x,T_y$可能不同     
做法：读完句子以后在开始逐个输出（encoder，decoder）     

<img style="float: center;" src="course_5_pics/WEEK_1_5.PNG" width=600 height=600>      

> ## 语言模型和序列生成     

假设在做一个语音识别系统，识别出两句非常相似的话，要判断那句话是正确的，在这里需要一个语言模型来判断两句话的出现概率。

### 语言模型     
- 训练集：一个语料库         
- 得到一个句子，首先把句子进行Tokenize（切分）后进行one-hot编码；    
- 判断句子结束时通常添加一个额外的标记```<EOS>```      
- 句子中的未知词替换成```<UNK>```未知词     
    
1. 第一个时间步:RNN读取零向量，输出$\hat y^{<1>}$。      
2. 下一个时间步:读取第一个词的正确结果(读取上一个时间步的结果,$x^{<t>}=y^{<t-1>}$)后在第一个词的条件概率下计算第二个词的概率；   
3. 在前继词语的条件下预测第三个词的概率
4. 最终建立一个**条件概率分布**判断序列的可能性     

<img style="float: center;" src="course_5_pics/WEEK_1_6.PNG" width=600 height=600> 

> ## 新序列采样     
         
训练出一个序列模型后，当需要确认该模型学到了什么，可以使用**新序列采样**的方法        

1. 已有一个训练好的语言模型/序列模型
2. 在第一个时间步输入$a^{<0>}=0$与$x^{<0>}=0$，得到一个softmax层的结果    
3. 在softmax层的结果中根据每个单词的概率随机采样(```np.random.choice```)，输入下一个时间步    
4. 下一个时间步输出的结果输入下下个时间步，重复直到出现```<EOS>```    
5. 当遇到```<UNK>```时可以在此进行一次采样，直到不是```<UNK>```为止

> ## 梯度消失与梯度爆炸    
```
例子:
1. The cats ,which already ate......,was full
2. The cats ,which already ate......,were full
如何判断was和were
```
- 在反向传播过程中需要对激活han函数进行求导，如果导数大于1，那么随着网络层数的增加梯度更新将会朝着指数爆炸的方式增加这就是梯度爆炸。同样如果导数小于1，那么随着网络层数的增加梯度更新信息会朝着指数衰减的方式减少这就是梯度消失。

- 激活函数在不同输入值区间的倒数不同

**基本RNN的缺点**：    
RNN难以学习长期依赖的结果——前向传播以及反向传播时，前后端很难相互影响(长句子中，前端时间步的输入结果很难影响到后端，后端时间步词语的反向传播很难影响到前端时间步神经元的权重)      

大型**深度**神经网络会在参数前后传播的过程中出现**指数级增长或减小**。                          

- RNN出现梯度爆炸——Gradient clipping解决
- 出现梯度消失——GRU，LSTM等



> ## GRU(Gate Recurrent Unit)——门控循环单元       
### 简单的GRU
- $t$时间的激活函数输出：$a^{<t>}$       
- 有一个memory cell，提供记忆的功能——在时间t时的记忆:$c^{<t>}=a^{<t>}$           
- $c^{<t>}$的估计值：$\tilde{c}^{<t>} = \tanh(w_u[c^{<t-1>},x^{<t>}]+b_c)$         
- 有一个Gate(0-1之间):$\Gamma u = \sigma(w_u[c^{<t-1>},x^{<t>}])$作用就是什么时候更新cell的值 ($\tilde{c}^{<t>}$代替$c^{<t>}$)        
- $c^{<t>} = \Gamma u * \tilde{c}^{<t>}+(1-\Gamma u)*c^{<t-1>}$

（$c^{<t>}$和$\Gamma u$的维度一样）

```假设一个句子：    
The cat,which already ate...,was full.
```
Gate会在```cat```处更新cell为1，代表谓语应该是单数形式，维持cell的值直到```was full```，即在```was```之前所有地方，Gate应该维持关闭。     

### 完整的GRU         
- $t$时间的激活函数输出：$a^{<t>}$       
- 有一个memory cell，提供记忆的功能——在时间t时的记忆:$c^{<t>}=a^{<t>}$           
- $c^{<t>}$的估计值：$\tilde{c}^{<t>} = \tanh(w_u[\Gamma_r c^{<t-1>},x^{<t>}]+b_c)$     
- $\Gamma_r = \sigma(w_u[c^{<t-1>},x^{<t>}]+b_r)$(相关性)
- 有一个Gate(0-1之间):$\Gamma u = \sigma(w_u[c^{<t-1>},x^{<t>}]+b_u)$作用就是什么时候更新cell的值 ($\tilde{c}^{<t>}$代替$c^{<t>}$)        
- $c^{<t>} = \Gamma u * \tilde{c}^{<t>}+(1-\Gamma u)*c^{<t-1>}$

（$c^{<t>}$和$\Gamma u$的维度一样）



> ## LSTM   

LSTM解决梯度消失——传统RNN是乘性
LSTM对记忆的操作**是相加的，线性的**，使得不同时序的记忆对当前的影响相同，为了让不同时序的记忆对当前影响变得可控，LSTM引入了输入门和输出门，之后又有人对LSTM进行了扩展，引入了遗忘门。

相比于GRU，LSTM中$c^{<t>}\neq a^{<t>}$ 且增加了两个门    
- $t$时间的激活函数输出：$a^{<t>}$       
- memory cell:$c^{<t>}=a^{<t>}$           
- $c^{<t>}$的估计值：$\tilde{c}^{<t>} = \tanh(w_u[a^{<t-1>},x^{<t>}]+b_c)$     
- update gate:$\Gamma u = \sigma(w_u[a^{<t-1>},x^{<t>}]+b_u)$
- forget gate:$\Gamma f = \sigma(w_f[a^{<t-1>},x^{<t>}]+b_f)$
- output gate:$\Gamma o = \sigma(w_o[a^{<t-1>},x^{<t>}]+b_o)$       
- $c^{<t>} = \Gamma u * \tilde{c}^{<t>}+\Gamma f*c^{<t-1>}$  
- $a^{<t>} = \Gamma o*c^{<t>}$ 
<img style="float: center;" src="course_5_pics/WEEK_1_7.PNG" width=600 height=600> 

常用版本
peehole connect（偷窥孔连接）——门值也可能受到$c^{<t-1>}$的影响   


> ## Bidirectional RNN（BRNN）     
```
He said,"Teddy bears are on sale"
在上述句子中，无法根据前三个单词判断Teddy是否是个人名
```      
无论是DNN,RNN,CNN都是单向传播的。   
<img style="float: center;" src="course_5_pics/WEEK_1_8.PNG" width=600 height=600>   
如图所示，BRNN的前向传播过程是由1-2-3-4，$x^{<4>}$输入时计算反向的$a^{<4>}$同时输出$\hat y^{<4>}$最后往回计算一直到$\hat y^{<1>}$      

> ## DRNN     

符号:$a^{[l]}{<t>}$代表第$l$层的第$t$个时间步的激活值    
<img style="float: center;" src="course_5_pics/WEEK_1_9.PNG" width=600 height=600> 